In [ ]:
!gdown --id 1baHMGphE705ikbV8hCk5_3FtanZ_dpSt

Downloading...
From: https://drive.google.com/uc?id=1baHMGphE705ikbV8hCk5_3FtanZ_dpSt
To: /content/aligned.tar.gz
2.81GB [00:39, 70.6MB/s]


In [ ]:
!gdown --id 1RR1LkayEgPTJql5-ITQmtsoTom9NRoOH

Downloading...
From: https://drive.google.com/uc?id=1RR1LkayEgPTJql5-ITQmtsoTom9NRoOH
To: /content/train_val_txt_files_per_fold.zip
3.11MB [00:00, 100MB/s]


In [ ]:
!unzip -n -q train_val_txt_files_per_fold.zip

In [ ]:
!tar -xf aligned.tar.gz

In [ ]:
!pip install pytorch-lightning
!pip install comet-ml
!pip install local-attention
!pip install einops

In [ ]:
from hparams import hyperparams
hyperparams.path_to_files

'/content'

In [ ]:
from depen import *
from model import Age_Gender_pl
from datasets import Adience_pl_dataset

from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [ ]:
class CheckpointEveryNSteps(pl.Callback):
    """
    Save a checkpoint every N steps, instead of Lightning's default that checkpoints
    based on validation loss.
    """

    def __init__(
        self,
        save_step_frequency,
        prefix="N-Step-Checkpoint",
        use_modelcheckpoint_filename=False,
    ):
        """
        Args:
            save_step_frequency: how often to save in steps
            prefix: add a prefix to the name, only used if
                use_modelcheckpoint_filename=False
            use_modelcheckpoint_filename: just use the ModelCheckpoint callback's
                default filename, don't use ours.
        """
        self.save_step_frequency = save_step_frequency
        self.prefix = prefix
        self.use_modelcheckpoint_filename = use_modelcheckpoint_filename

    def on_batch_end(self, trainer: pl.Trainer, _):
        """ Check if we should save a checkpoint after every train batch """
        epoch = trainer.current_epoch
        global_step = trainer.global_step
        if global_step % self.save_step_frequency == 0:
            if self.use_modelcheckpoint_filename:
                filename = trainer.checkpoint_callback.filename
            else:
                filename = f"{self.prefix}_{epoch}_{global_step}.ckpt"
            ckpt_path = os.path.join(trainer.checkpoint_callback.dirpath, filename)
            trainer.save_checkpoint(ckpt_path)

In [ ]:
seed_e(42)

In [ ]:
comet_logger = CometLogger(
    save_dir='/content/log/',
    api_key="23CU99n7TeyZdPeegNDlQ5aHf",
    project_name="dl-proj",
    workspace="etzelkut",
    # rest_api_key=os.environ["COMET_REST_KEY"], # Optional
    # experiment_name="default" # Optional
)

CometLogger will be initialized in online mode


In [ ]:
dataset_pl = Adience_pl_dataset(hyperparams)
dataset_pl.prepare_data()
dataset_pl.setup()
train_loader = dataset_pl.train_dataloader()
steps_per_epoch = int(len(train_loader))
print(steps_per_epoch)

already prepared but later add
ready
198


In [ ]:
checkpoint_callback = ModelCheckpoint(
                                      monitor='val_loss',
                                      save_last=True, 
                                      dirpath= "/content/drive/MyDrive/dl_proj/gg/",#os.path.join(path, "/checkpoints"),
                                      filename='samplemodel{epoch}.ckpt',
                                      save_top_k=3,
                                      mode='min',
                                      )
every_epoch = CheckpointEveryNSteps(save_step_frequency = steps_per_epoch, use_modelcheckpoint_filename = True)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory /content/drive/MyDrive/dl_proj/gg/ exists and is not empty. With save_top_k=3, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)


In [ ]:
model = Age_Gender_pl(hyperparams, steps_per_epoch = steps_per_epoch)
#lr_monitor = LearningRateMonitor(logging_interval='step')

performer
glu


In [ ]:
trainer = Trainer(callbacks=[checkpoint_callback, every_epoch], #lr_monitor],
                    logger=comet_logger,
                    gpus=1,
                    profiler=True,
                    #auto_lr_find=True, #set hparams
                    #gradient_clip_val=0.5,
                    check_val_every_n_epoch=2,
                    #early_stop_callback=True,
                    max_epochs = hyperparams.epochs,
                    #min_epochs=400,
                    progress_bar_refresh_rate = 0,
                    deterministic=True,)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
trainer.fit(model, dataset_pl)

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/etzelkut/dl-proj/d741d2b4f4144166b6397e633901dfa0


  | Name    | Type             | Params
---------------------------------------------
0 | network | Model_dl         | 32.3 M
1 | loss    | CrossEntropyLoss | 0     
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


In [ ]:
checkpoint_name = os.path.join("/content/drive/MyDrive/dl_proj/gg", 'manual_save' + str(228) + '.ckpt')
trainer.save_checkpoint(checkpoint_name)

In [ ]:
trainer.test()

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/etzelkut/dl-proj/a7a31a53fcc644da9c2609c515175e88

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Your test_dataloader has `shuffle=True`, it is best practice to turn this off for validation and test dataloaders.
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'epoch_now': 58,
 'test_acc': tensor(28.1821, device='cuda:0'),
 'test_loss': tensor(1.9430, device='cuda:0'),
 'train_acc': tensor(55.2302, device='cuda:0'),
 'train_loss': tensor(1.3213, device='cuda:0'),
 'train_loss_epoch': tensor(1.1964, device='cuda:0'),
 'train_loss_step': tensor(1.3213, device='cuda:0'),
 'val_loss': tensor(1.2991, device='cuda:0'),
 'vall_acc': tensor(51.7145, device='cuda:0')}
--------------------------------------------------------------------------------


[{'epoch_now': 58,
  'test_acc': 28.182090759277344,
  'test_loss': 1.942962884902954,
  'train_acc': 55.23016357421875,
  'train_loss': 1.3213006258010864,
  'train_loss_epoch': 1.1964330673217773,
  'train_loss_step': 1.3213006258010864,
  'val_loss': 1.2991019487380981,
  'vall_acc': 51.71448516845703}]